In [1]:
import pandas as pd
import numpy as np

import pymorphy2
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

In [2]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/jupyter/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/jupyter/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /home/jupyter/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /home/jupyter/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /home/jupyter/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nlt

True

In [3]:
morph = pymorphy2.MorphAnalyzer()
stop_words = set(stopwords.words('russian'))

In [4]:
data_without_problem = []
with open('JOB_LIST.csv', 'r', encoding='utf-8', errors='replace') as file:
    for line in file:
        sentence = line
        sentence = sentence.replace('-', ' ').replace('.', ' ').replace(',', ' ').replace('(', ' ').replace(')', ' ').replace(';', ' ').replace(':', ' ')
        sentence = sentence.replace('«',' ').replace('»',' ').replace('"',' ').replace("'",' ').replace('\n', '').replace('+',' ')
        sentence = sentence.replace('1',' ').replace('2',' ').replace('3',' ').replace('4',' ').replace('5',' ').replace('6',' ').replace('7',' ').replace('8',' ').replace('9',' ')
        item_of_array = [sentence]
        data_without_problem.append(item_of_array)

In [5]:
columns = ['job_title']
data = pd.DataFrame(data=data_without_problem, columns=columns)
data.drop(data[data['job_title'] == 'job_title'].index, inplace=True)

unique_data = data.drop_duplicates()
unique_array = unique_data.to_numpy()
print(unique_data)

                                                  job_title
1          Старший инспектор отдела пособий субсидий и к...
2         Инспектор отдела муниципальных закупок и имуще...
3                                            Ведущий геолог
4                                        Геолог   категории
6                                  Начальник горного отряда
...                                                     ...
11740841                             Практикант  архивариус
11740873                                Дизайнер версталщик
11740959  Электрослесарь подземный   разряда   группа до...
11740981     Юрист по защите интеллектуальной собственности
11740987  Инженер по нор�                               ...

[965240 rows x 1 columns]


In [6]:
filtered_data = []
for i in range(len(unique_array)):
    tokens = word_tokenize(unique_array[i][0])
    lemmatized = [morph.parse(word)[0].normal_form for word in tokens if word.isalpha()]
    filtered = [word for word in lemmatized if word not in stop_words]
    filtered_data.append(filtered)

In [7]:
#Удаление глагола и прилагательного
clear_data = []
for i in range(len(filtered_data)):
    update_prof = []
    for j in range(len(filtered_data[i])):
        word = filtered_data[i][j]
        parsed_word = morph.parse(word)[0]
        pos = parsed_word.tag.POS
        if pos not in {'VERB', 'INFN', 'ADJF', 'ADJS'}:
            update_prof.append(word)
    clear_data.append(update_prof)
            

In [8]:
full_prof_data = []
for i in range(len(clear_data)):
    prof = ' '.join(clear_data[i])
    full_prof_data.append([prof])

In [9]:
full_prof_df = pd.DataFrame(data=full_prof_data, columns=columns)

unique_df = full_prof_df.drop_duplicates()
unique_df_array = unique_df.to_numpy()
print(unique_df)

                                                job_title
0       старший инспектор отдел пособие субсидия компе...
1                       инспектор отдел закупка отношение
2                                                  геолог
3                                        геолог категория
4                                         начальник отряд
...                                                   ...
965233                 инженер отделение терапия институт
965234                    практикант оператор ввод данные
965236                                дизайнер версталщик
965237                электрослесарь разряд группа допуск
965238                         юрист защита собственность

[643086 rows x 1 columns]


In [10]:
pip install transformers

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [11]:
from transformers import AutoTokenizer, AutoModel
import torch

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [12]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")

/home/jupyter/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.

In [40]:

inputs = tokenizer(unique_df_array[0][0], return_tensors="pt", padding=True, truncation=True)
with torch.no_grad():
    outputs = model(**inputs)
embeddings = outputs.last_hidden_state.mean(dim=1)

print(embeddings)



tensor([[ 2.1605e-02, -1.4460e-01,  6.9122e-01, -1.6144e-01,  3.5140e-01,
         -8.1669e-02,  1.0708e-02, -1.2299e-01,  4.7647e-02,  4.6301e-01,
          9.8157e-02,  4.1878e-01, -1.1369e-01,  9.1312e-02, -1.9702e-01,
         -3.7058e-01,  3.6003e-01,  4.4512e-01,  1.5548e-02,  1.1922e-01,
         -1.2296e-01,  4.7439e-01, -5.2710e-02,  2.3751e-01,  2.6807e-01,
          5.5393e-01, -8.0457e-02,  1.8410e-01,  2.9570e-02, -4.8771e-02,
          4.4920e-01,  3.8477e-01, -4.0670e-01,  1.3456e-02, -6.5556e-02,
         -1.1390e-01, -1.7016e-01,  5.0739e-02,  3.9336e-02,  1.8408e-01,
         -4.0277e-02,  2.0189e-01,  9.3910e-02,  7.9964e-02, -6.0466e-02,
         -2.1813e-01, -5.6273e-01, -1.7290e-01, -3.8191e-01,  6.4315e-01,
         -7.1921e-02, -3.6235e-01, -2.0458e-03, -6.7839e-01, -3.2542e-01,
          1.8468e-02,  1.8058e-01,  2.7115e-01, -2.2644e-02,  3.8999e-01,
         -1.5301e-01, -4.5179e-01, -6.1599e-02, -7.8422e-02, -3.5033e-01,
          2.9691e-01,  4.1406e-01,  1.